In [89]:
import numpy as np
import math

In [82]:
train = np.loadtxt("pa2train.txt")
test = np.loadtxt("pa2test.txt")
validation = np.loadtxt("pa2validation.txt")

In [235]:
node_count = 0

In [337]:
class Node:
    def __init__(self):
        self.data = []
        # less than label?
        self.less_eq = None
        self.greater = None
        self.label = None
        self.decision = None
        global node_count
        node_count += 1

In [338]:
def id_three(train):
    root = Node()
    root.data = train 
    impure = []
    impure.append(root)
    
    
    while impure:
        n = impure[0]
        impure.pop(0)
        
        if pure(n):
            n.label= n.data[0][22]
        else:
            impure.extend(list(split_decision(n)))
        
    return root
    

In [339]:
train[:10,0].argsort()

array([0, 7, 6, 3, 5, 1, 2, 4, 8, 9], dtype=int64)

In [340]:
def split_decision(n):
    n.less_eq = Node()
    n.greater = Node()
    

    d = n.data
    total = len(d)
    # tuple (category, threshold)
    most_gain = ()
    index = -1
    # essentially infinite
    ent = np.inf
    
    
    # find the best split
    for cat in range(22):
        # sort the data according category
        c = d[d[:,cat].argsort()]

        for x in range(len(c)-1):
            # data points with the same value are skipped over
            if c[x][cat] == c[x+1][cat]:
                continue
            
            thresh = (c[x][cat] + c[x+1][cat])/2
            b = ((x+1)/total)
            t_ent = (b)*entropy(c[:x+1]) + (1-b)*entropy(c[x+1:])
            
    
            if t_ent < ent:
                ent = t_ent
                most_gain = (cat, thresh)
                index = x + 1
    
    # create child nodes
    sorted_d = d[d[:,most_gain[0]].argsort()]
    n.less_eq.data = sorted_d[:index]
    n.greater.data = sorted_d[index:]
    
    # update parent
    n.decision = most_gain
    
    return n.less_eq, n.greater

In [341]:
def entropy(data):
    pos = 0
    neg = 0
    total = len(data)
    for x in data:
        if x[22] == 1:
            pos += 1
        else:
            neg += 1

    if pos == 0 or neg == 0:
        return 0
        
    ret = (pos/total)*math.log(pos/total)+(neg/total)*math.log(neg/total)
    return -ret   

In [342]:
def pure(node):
    l = node.data[0][22]
    
    for i in node.data:
        if l != i[22]:
            return False
    return True 
        

In [343]:
from time import clock
timer = clock() 
node_count =  0
tree = id_three(train)
print(clock() - timer)

45.14534327069305


In [317]:
p = tree.less_eq.greater.data
p = p[p[:,20].argsort()]

In [312]:
p[p[:,20].argsort()][23][20] + p[p[:,20].argsort()][24][20]

25401.0

In [313]:
p[p[:,20].argsort()][3][20] + p[p[:,20].argsort()][4][20]

416.0

In [305]:
tree.less_eq.greater.decision

(0, 415000.0)

In [345]:
print(node_count)

405


In [267]:
p[p[:,20].argsort()]

array([[  5.00000000e+05,   1.00000000e+00,   0.00000000e+00,
          1.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   3.26470000e+04,   1.83200000e+03,
          1.92000000e+03,   2.17340000e+04,   1.83700000e+03,
          9.60000000e+01,   1.83200000e+03,   1.92000000e+03,
          2.17340000e+04,   1.83700000e+03,   0.00000000e+00,
          1.96370000e+04,   1.00000000e+00],
       [  5.00000000e+05,   2.00000000e+00,   1.00000000e+00,
          1.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   6.69400000e+03,   1.74700000e+04,
          1.50910000e+04,   1.58700000e+03,   6.13850000e+04,
          5.62520000e+04,   1.74700000e+04,   1.50910000e+04,
          1.58700000e+03,   6.13850000e+04,   0.00000000e+00,
          5.67330000e+04,   1.00000000e+00],
       [  4.50000000e+05,   2.00000000e+00

In [318]:
three_ent = (4/35)*entropy(p[:4]) + (1-(4/35))*entropy(p[4:])

tt_ent = (24/35)*entropy(p[:24]) + (1-(24/35))*entropy(p[24:])
print(three_ent)
print(tt_ent)

0.573198358007157
0.5710436972651516


In [344]:
tree.less_eq.greater.decision

(20, 208.0)